<a href="https://colab.research.google.com/github/matheus97eng/desafio_solvimm/blob/main/notebook_de_apresentacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Apresentação do problema

O problema trabalhado neste notebook se trata de um desafio proposto em um processo seletivo. O desafio é descrito a seguir:

A empresa "Ponto Quente" está enfrentando um problema no seu banco de dados. De alguma forma, seus produtos foram categorizados de forma errada e isso precisa ser corrigido. O time de inteligência de dados da empresa conseguiu montar um dataset contendo alguns produtos com a classificação correta. Nesse dataset, encontramos várias informações sobre o produto, inclusive os reviews dos clientes.

A empresa "Ponto Quente" deseja um modelo de machine learning que seja capaz de categorizar automaticamente os seus produtos, baseado nas informações dos reviews dos clientes. Mais especificamente, a empresa quer um programa que recebe o modelo treinado junto com um dataframe a ser categorizado. Este dataframe não pode possuir a coluna `product_category`, que é a coluna que fornece a informação da categoria dos produtos. O programa a ser desenvlvido deve retornar o mesmo dataframe de entrada, mas agora com a coluna `product_category`, com as classificações previstas pelo modelo.

Para resolver esse problema, foi utilizado o algoritmo de [Naive Beyes](https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/), do tipo Multinomial, que obteve uma acurácia de aproximadamente 73,24%.

O tratamento dos dados, preparação, treinamento e validação do modelo foi toda baseada nesses dois notebooks:

+ https://github.com/AarohiSingla/Multinomial-Naive-Bayes/blob/master/news_classifier_unseen_input.ipynb
+ https://github.com/AarohiSingla/Multinomial-Naive-Bayes/blob/master/youtube_multinomial_naive_bayes.ipynb

Outras consultas foram feitas na documentação das bibliotecas e, sobre remoção de emojis, foi consultado essa [dúvida do stackoverflow](https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python).

## importando bibliotecas

No próximo código importamos as bibliotecas e métodos utilizados para resolução do desafio. Cada uma delas será descrita abaixo também.

In [1]:
import pandas as pd
import string, re, nltk
#from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

nltk.download('stopwords')      # obtem as stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- pandas - manipulação de dataframes
- nltk.word_tokenize - 
- nltk.corpus.stopwords - lista de palavras consideradas "stopwords"
- string - manipulação de strings
- re - biblioteca python para operação de strings com codificação. No caso iremos trabalhar com a codificação dos emojis.
- da biblioteca sklearn:
    - MultinomialMB - algoritmo de Naive Beyes do tipo multinominal, o mais indicado para multiclassificação
    - train_test_split - divide os dados em treino e teste
    - accuracy_score - método utilizado para validar o modelo

## funções utilizadas no notebook

Para ajudar na leitura do notebook e minimizar as linhas de código durante a apresentação, serão feclaradas aqui no início as funções utilizadas na apresentação, inclusive a função `validate` pedida no desafio. O funcionamento dessas funções será explicado ao longo das apresentações.

In [15]:
def tratamento_reviews(df):
    
    # juntando o conteúdo das duas colunas de texto dos reviews
    chars = [(df['review_headline'].iloc[i] + ' ' + df['review_body'].iloc[i])  
             for i in df.index.to_list()]
    df['review_total'] = chars

    # preparando uma lista de emojis a serem excluídos 
    emoji_pattern = re.compile("["      
        u"\U0001F300-\U0001F5FF"  # símbolos
        u"\U0001F680-\U0001F6FF"  # transporte e símbolos de mapa
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0000231A-\U000023F3"  # relógios e setas
        u"\U000026A1-\U000026BE"  # relâmpago, cores e bolas de esportes
        u"\U00002753-\U00002757"  # pontuação
        u"\U00002B50"             # estrela
        u"\U0001F32D-\U0001F37F"  # comidas
        u"\U0001F3A0-\U0001F3D3"  # esportes
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F910-\U0001F93E"  # mais emoticons e emojis de esportes
                           "]+", flags=re.UNICODE)
    
    # preparando para excluir os caracteres indesejados
    col_corrigida = []      # lista para armazenar o conteúdo já tratado e a ser colocado na coluna review_total
    char_excluir = string.punctuation + string.digits       # lista contendo caracteres a serem excluídos: caracteres escpeciais e dígitos

    for row in df['review_total']:
        temp = [char for char in row if char not in char_excluir]   # excluindo digitos e caracteres especiais
        text = ''.join(temp).lower()
        text = emoji_pattern.sub(r'', text)
        for word in stopwords.words('english'):
            text.replace(word, '')
        col_corrigida.append(text)

    df['review_total'] = col_corrigida

    return df

def tratamento_categorias(df):

    df['num_category'] = df['product_category'].map({'Digital_Ebook_Purchase':0, 
       'Music':1, 'Video DVD':2, 'Mobile_Apps':3, 'Books':4, 'Electronics':5, 
       'Toys':6, 'Video Games':7, 'Digital_Video_Download':8, 'Digital_Music_Purchase':9, 
       'PC':10, 'Camera':11, 'Baby':12, 'Wireless':13, 'Home Entertainment':14, 'Sports':15,
       'Musical Instruments':16, 'Lawn and Garden':17, 'Home Improvement':18, 'Home':19, 
       'Watches':20, 'Video':21, 'Shoes':22, 'Office Products':23, 'Automotive':24, 
       'Health & Personal Care':25, 'Personal_Care_Appliances':26, 'Software':27, 
       'Kitchen':28, 'Luggage':29, 'Pet Products':30, 'Beauty':31})
    
    return df

def split_and_vect(df_ML, seed):
    
    x_train, x_test, y_train, y_test = train_test_split(df_ML['review_total'], df_ML['num_category'], random_state=seed)

    vect = CountVectorizer(ngram_range=(2,2))
    X_train = vect.fit_transform(x_train)
    X_test = vect.transform(x_test)

    return X_train, X_test, y_train, y_test, vect

def validate(modelo, vect, df_teste):

    df_tratado = tratamento_reviews(df_teste)       # tratando os textos de reviews

    texto_vetorizado = vect.transform(df_tratado['review_total'])

    df_tratado['product_category'] = modelo.predict(texto_vetorizado)  # realizando predição do modelo e atribuindo a uma nova coluna do df

    # por fim, precisamos transformar de volta os tokers numéricos nas classes originais
    df_tratado['product_category'] = df_tratado['product_category'].map({0:'Digital_Ebook_Purchase', 
       1:'Music', 2:'Video DVD', 3:'Mobile_Apps', 4:'Books', 5:'Electronics', 
       6:'Toys', 7:'Video Games', 8:'Digital_Video_Download', 9:'Digital_Music_Purchase', 
       10:'PC', 11:'Camera', 12:'Baby', 13:'Wireless', 14:'Home Entertainment', 15:'Sports',
       16:'Musical Instruments', 17:'Lawn and Garden', 18:'Home Improvement', 19:'Home', 
       20:'Watches', 21:'Video', 22:'Shoes', 23:'Office Products', 24:'Automotive', 
       25:'Health & Personal Care', 26:'Personal_Care_Appliances', 27:'Software', 
       28:'Kitchen', 29:'Luggage', 30:'Pet Products', 31:'Beauty'})

    return df_tratado.drop('review_total', axis=1)

## obtenção dos dados e tratamento

Os dados foram fornecidos pela empresa SOLVIMM, que propôs o desafio. Eles correspondem a uma base de dados de produtos cadastrados pela empresa "Ponto Quente".

In [3]:
arq = 'https://github.com/matheus97eng/desafio_solvimm/blob/main/data/reviews.tsv?raw=true' # repositório do github
df_original = pd.read_csv(arq, sep='\t')
print(df_original.shape)
df_original.head()

(170583, 16)


,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,product_category
0,762868,UK,29723892,R3VNENIATVV8QE,B00NOPQU2K,627793267,The Girl on the Train,5,0,1,N,N,Gripping you right where it matters,I know to say a story is &#34;gripping&#34; is...,2015-04-27,Digital_Ebook_Purchase
1,1284183,UK,41072087,R2U3LV67N99770,B0013F2LSK,13214624,11,5,2,5,N,Y,The Best of Me,"This album is totally fantastic, a great mix o...",2008-03-18,Music
2,1599315,UK,49938094,R3RO94POCHNI9V,B005CVWWJY,769273676,Ready Player One,5,0,0,N,Y,superb,Enjoyed every second of this book. It took me...,2014-08-28,Digital_Ebook_Purchase
3,204782,UK,14398213,R3S2BB5SBWBC1,B00008AWV3,841759677,The Four Feathers [DVD] [1939],5,1,1,N,Y,"Sweeping, authentic historic drama",I loved the historic scenes---the English coun...,2013-12-27,Video DVD
4,352938,UK,20140500,R27E2PNXJSWJIN,B00FAXJHCY,803172158,The Martian,4,0,0,N,Y,... a few pages to get through it but a good b...,May have skipped a few pages to get through it...,2015-06-06,Digital_Ebook_Purchase


### Informações das features, exclusão de dados nulos e explicação do modelo:

Vamos obter uma visão geral das features em questão.

In [4]:
display(df_original.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170583 entries, 0 to 170582
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         170583 non-null  int64 
 1   marketplace        170583 non-null  object
 2   customer_id        170583 non-null  int64 
 3   review_id          170583 non-null  object
 4   product_id         170583 non-null  object
 5   product_parent     170583 non-null  int64 
 6   product_title      170583 non-null  object
 7   star_rating        170583 non-null  int64 
 8   helpful_votes      170583 non-null  int64 
 9   total_votes        170583 non-null  int64 
 10  vine               170583 non-null  object
 11  verified_purchase  170583 non-null  object
 12  review_headline    170583 non-null  object
 13  review_body        170582 non-null  object
 14  review_date        170578 non-null  object
 15  product_category   170583 non-null  object
dtypes: int64(6), object(

None

O desafio pede para classificar os produtos somente de acordo com os reviews feitos pelos clientes. Esclarecido isso, não precisamos nos preocupar com outras features, a não ser: `review_headline`, que é o título da avaliação, `review_body`, que é a avaliação em si e por fim, `product_category`, que é a categorização corrigida do produto. Aqui não consideraremos relevante a data de postagem do review, nem o ID do review.

Dado que as features `review_headline` e `product_category` são todas texto (do tipo caracter), e por se tratar de um problema de classificação (identificar qual a classe que o produto pertence), precisamos de um modelo que utilize NPL (Natural Language Processing). Será escolhido o algoritmo de [Naive Beyes](https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/), pois é um algoritmo de fácil aplicação com python e sklearn, tem uma boa resposta dado uma quantidade pequena de dados e não é um algoritmo pesado. 

Explicando de uma forma não formal, o algoritmo de Nave Beyes trabalha calculando qual é a probabilidade de um produto ser da classe "X" sendo que no review desse produto encontramos certas "palavras-chave". Treinando o modelo, ele consegue identificar, por exemplo que, se no review aparecer a palavra "bola", é mais provável que o produto seja da categoria "esportes". Com as informações da base de dados, o algoritmo estará treinado e validado para classificar outros produtos, fora da base de dados preparada pelo time de dados da "Ponto Quente". 

Uma desvantagem desse modelo é que ele não considera a semântica do texto. Por exemplo, ao olhar para a frase "meus dedos ficaram muito apertados quando eu testei na corrida". Analisando o contexto, poderíamos identificar que o produto se trata provavelmente de um tênis. Mas o Nave Beyes analisa palavra por palavra, separadamente, o que tornaria mais difícil a identificação, nesse exemplo dado.

No entanto, antes de modelar nosso problema, precisamos tratar os nossos dados.
Executando `df_original.info()` já vemos que a feature `review_body` possui uma linha com dado nulo. Isso porque vemos 170582 dados não nulos nessa coluna, enquanto que a nossa base de dados possui 170583 linhas. Vamos optar por excluir toda a linha que contém esse dado nulo, já que atrapalhará no desenvolvimento do modelo e se trata apenas de um produto.

In [5]:
excluir = df_original[df_original['review_body'].isnull()].index
df_tratado = df_original.drop(excluir).reset_index(drop=True)
df_tratado.head()

,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,product_category
0,762868,UK,29723892,R3VNENIATVV8QE,B00NOPQU2K,627793267,The Girl on the Train,5,0,1,N,N,Gripping you right where it matters,I know to say a story is &#34;gripping&#34; is...,2015-04-27,Digital_Ebook_Purchase
1,1284183,UK,41072087,R2U3LV67N99770,B0013F2LSK,13214624,11,5,2,5,N,Y,The Best of Me,"This album is totally fantastic, a great mix o...",2008-03-18,Music
2,1599315,UK,49938094,R3RO94POCHNI9V,B005CVWWJY,769273676,Ready Player One,5,0,0,N,Y,superb,Enjoyed every second of this book. It took me...,2014-08-28,Digital_Ebook_Purchase
3,204782,UK,14398213,R3S2BB5SBWBC1,B00008AWV3,841759677,The Four Feathers [DVD] [1939],5,1,1,N,Y,"Sweeping, authentic historic drama",I loved the historic scenes---the English coun...,2013-12-27,Video DVD
4,352938,UK,20140500,R27E2PNXJSWJIN,B00FAXJHCY,803172158,The Martian,4,0,0,N,Y,... a few pages to get through it but a good b...,May have skipped a few pages to get through it...,2015-06-06,Digital_Ebook_Purchase


### Features de review dos clientes

Há duas colunas no dataframe com o conteúdo de avaliações dos clientes: `review_headline`, que contém o título da avaliação, e `review_body`, que contém o corpo do review. O que será feito nesse desafio será juntar todas as palavras dessas duas features em uma coluna apenas, já que o contexto do texto não importa no modelo de Naive Beyes. O nome da coluna contendo o texto compactado será `review_total`.

Além disso, precisamos fazer a limpeza desse texto. Será tirado todos os caracteres especiais, dígitos e uma lista de emojis. Além disso, serão removidas as chamadas **"stop-words"**, que são basicamente palavras que não nos fornecem muita informação quando analisadas separadamente. São palavras como "I, yourself, the...". A biblioteca `ntlk` possui uma lista dessas palavras. Será usado essa lista como base. Feito a limpeza, garantimos o melhor funcionamento do modelo, que analisará apenas palavras-chave dos dados.

**obs.:** não serão excluídos todos os emojis possíveis. A lista de todos os emojis codificados é muito grande e poderia fazer o programa demorar muito para ser executado. Foram escolhidos emojis que são mais prováveis de aparecer em produtos. A lista de emojis a serem excluídos pode ser editada na própria função `tratamento_review`, que faz a limpeza dos textos dos reviews.

### Tipos de classificação

É importante também olharmos para os tipos de classificação de produtos que a empresa tem. Na base fornecida, foram identificados 32 classes. É essencial entender que o modelo a ser treinado **não identificará classes de produtos que não estão na base de dados**. Desse modo, se a empresa quisesse identificar um produto como classe "carro" ou não, teria que acrescentar à base de dados vários produtos da categoria "carro".

O modelo de machine learning não consegue interpretar dados do tipo string. Portanto, precisamos alterar os dados da coluna alvo `product_category`, fazendo uma tokerização simples, em outras palavras, substituindo as palavras por números. Os valores substituídos serão armazenados em uma coluna chamada `num_category`

### execução das funções de tratamento do dataframe

Todo esse tratamento descrito acima será feito por duas funções: `tratamento_reviews`, que tratará todo o conteúdo dos reviews dos clientes e `tratamento_categorias`, que tratará o conteúdo da coluna `product_category`. Enquanto que a primeira função retorna o dataframe acrescentado da coluna `review_total` (coluna esta que é criada pela próppria função), a segunda função reotornará o dataframe com os dados preparados para ser desenvolvido o modelo. Esse dataframe será chamado `df_ML` e conterá a coluna `review_total`, que será a variável x do modelo, e a coluna `num_category`, que será a variável y.

In [6]:
df_tratado = tratamento_reviews(df_tratado)
df_ML = tratamento_categorias(df_tratado)

## Aplicação do machine learning

### divisão dos dados em treino e teste / treinamento do modelo

Após os tratamentos feitos, vamos separar os dados em treino e teste para o modelo. No entanto, mais uma transformação deverá ser feita na coluna `review_total`. Precisamos fazer a tokerização (ou vetorização) das palavras, além de transformar a coluna em uma matriz esparça, que é a entrada que o método `fit` do modelo `MultinomialNB` aceita. Para isso utilizaremos a classe `CountVectorizer` da biblioteca `sklearn`. Aqui, os dados da vetorização e da transformação em matriz esparça serão armazenados nas variáveis `X_train` e `X_test` (com X maiúsculo). A transformação será feita em cima de `x_train` e `x_test` (com x minúsculo), variáveis que serão preparadas através do método `train_test_split`, também da biblioteca `sklearn`. Estas variáveis são apenas uma separação dos dados de `df_ML`.

Todo esse processo, bem como o treinamento do modelo, serão realizados pela função `split_and_vect`, que retornará as matrizes esparças `X_train` e `X_test` e os arrays `y_train` e `y_test`. Além dessas variáveis, a função retornará também a instância `vect`, que será utilizada na função `validate` para tokerizar as palavras dos textos. Os parâmetros que esta função recebe são o dataframe (que deve ser tratado pelas duas funções de tratamento) e o número `seed`, que garante a reprodutibilidade do modelo. Aqui executaremos a função com seed = 50.

In [7]:
X_train, X_test, y_train, y_test, vect = split_and_vect(df_ML, 50)

modelo = MultinomialNB(alpha=0.2)
modelo.fit(X_train,y_train)

result = modelo.predict(X_test)
print(result)

[2 2 3 ... 2 4 1]


### validação do modelo

Para validar o modelo, utilizaremos o proposto pelo desafio da Solvimm, que é calcular a acurácia. Isso será calculado através da biblioteca `sklearn`.

O modelo desenvolvido aqui apresenta uma acurácia de aproximadamente 73,24%. Em outras palavras, com a divisão dos dados feitas aqui e com esse modelo treinado, estamos acertando praticamente a classificação de 3 a cada 4 produtos. Essa é uma acurácia maior do que o mínimo esperado no desafio.

In [8]:
accuracy_score(result,y_test)

0.7324250808985603

## Aplicando o modelo: função validate

Por fim, após o tratamento, treinamento e validação do modelo, resta desenvolver uma função que aplique nosso modelo a um dataframe, afim de classificar os produtos nele contidos. Para isso, utilizaremos a função `validate`, que recebe o modelo treinado, a função vect e um dataframe como o fornecido pela equipe da "Ponto Quente", mas sem a coluna `product_category`. A função deve retornar o mesmo dataframe de entrada, porém com a coluna `product_category` que terá as categorias previstas para cada produto.

Não faz sentido nenhum executar esta função sobre a base de dados fornecida pela "Ponto Quente" para preparar o modelo, já que as classificações dos produtos já foram corrigidas pelo time da empresa. No entanto, utilizaremos a mesma base de dados apenas para verificar o funcionamento da função, uma vez que o modelo já está validado.

É importante dizer que, antes de fazer de fato a predição do modelo, o dataframe que a função `validate` recebe precisa ser tratado, assim como fizemos o tratamento dos textos dos reviews antes de treinar o modelo. Mais especificamente, o dataframe precisa passar antes pela função `tratamento_reviews`, o que leva boa parte do tempo de execução de `validate`. Além disso, dentro da função precisa ser feita a tokerização das palavras, por isso o parâmetro `vect` deve ser fornecido. Sem esse parâmetro, não teria como a empresa "Ponto Quente" usar a função `validate` sem executar antes 

O dataframe que será utilizado como parâmetro de execução da função será `df_tratado`, e não `df_original`, aquele retirado diretamente da base fornecida pelo time de dados. Isso porque **o dataframe que `validate` recebe não pode conter dados nulos nos reviews**. Não será implementado nenhuma linha de código para tratar dados nulos, porque isso seria melhor feito sendo conversado com a empresa. É fácil entender o porque: digamos que a empresa aplique o algoritmo numa dataframe com vários dados nulos. Ela gostaria que simplesmente ignorássemos os produtos que não tiveram reviews ou desejaria que fosse feito um outro tipo de tratamento no algoritmo? Cabe a empresa decidir.

In [ ]:
df_sem_categoria = df_tratado.drop(['product_category', 'review_total'], axis=1).sample(100).reset_index(drop=True)

df_categorizado = validate(modelo, vect, df_sem_categoria)

df_categorizado.head()

## Conclusões e possíveis ponto de melhoria

Por fim, considera-se como satisfatório o modelo desenvolvido para este desafio, que passou no critério de pontuação da acurácia do processo seletivo. Ele possui um tempo de execução bem aceitável, ainda mais dado o tamanho do dataset fornecido para treino e teste, que supera as 170 mil linhas. Isso não impede de serem estudadas algumas melhorias nas implementações e estruturação das funções.

Ainda poderia ser explorado os parâmetros do algoritmo `MultinomialNB`, para verificar as implicações e possíveis melhorias na acurácia do modelo.

No entanto, como dito, o algoritmo de Naive Beyes não considera contextualização de textos. Por isso, outros algoritmos como o BERT, que consideram interpretação de texto, possam apresentar uma pontuação de acurácia melhor. 

Como foi dito, foi também desconsiderado os emojis. Pode ser que usar os emojis como dados para o treinamento do modelo funcione melhor do que simplesmente ignorá-los. Por exemplo, um emoji de câmera pode dizer que, de fato, o produto que recebeu o review é uma câmera. Isso no entanto é algo que precisa de uma extensa base de dados contendo emojis, pois as pessoas os utilizam de maneiras diferentes com propósitos diferentes. 